In [1]:
## import libraries
import os, sys
import yaml
import xarray as xr
import pandas as pd
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units

%matplotlib inline

sys.path.append('../modules')
import ar_funcs

In [2]:
path_to_data = '/cw3e/mead/projects/cwp140/scratch/dnash/data/'      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
## read AR duration file
duration_df = pd.read_csv('../out/AR_track_duration_SEAK.csv')

In [4]:
duration_df

,Unnamed: 0,trackID,start_date,end_date,duration
0,0,2.000011e+11,2000-01-06 00:00:00,2000-01-06 18:00:00,18.0
1,1,2.000011e+11,2000-01-08 00:00:00,2000-01-08 06:00:00,6.0
2,2,2.000012e+11,2000-01-19 06:00:00,2000-01-19 12:00:00,6.0
3,3,2.000012e+11,2000-01-27 06:00:00,2000-01-31 00:00:00,90.0
4,4,2.000013e+11,2000-01-28 18:00:00,2000-01-29 00:00:00,6.0
...,...,...,...,...,...
1495,1495,2.019082e+11,2019-08-16 00:00:00,2019-08-21 12:00:00,132.0
1496,1496,2.019082e+11,2019-08-21 12:00:00,2019-08-22 06:00:00,18.0
1497,1497,2.019082e+11,2019-08-22 18:00:00,2019-08-23 00:00:00,6.0
1498,1498,2.019082e+11,2019-08-23 12:00:00,2019-08-24 06:00:00,18.0


In [ ]:
## load the precipitation datasets 



In [ ]:
## loop through each row in the AR duration df
## pull out the total accumulated preciptiation
## pull peak 1h precip rate for duration of event
## pull peak 3h precip rate for duration of event
## pull peak 6h precip rate for duration of event
## pull peak 12h precip rate for duration of event
## pull peak 24h precip rate for duration of event